In [ ]:
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from typing import TypedDict, Annotated
import random
import pprint

# Initialize the model
llm = ChatOllama(model="deepseek-r1:1.5b", temperature=0.7)

In [ ]:
class BookState(TypedDict):
    preferences: dict
    recommended_book: str
    feedback: str
    weights: dict

# Simulated book database
book_database = {
    "The Creative Mind": {"creativity": 9, "accuracy": 3, "knowledge": 6},
    "Science Facts 101": {"creativity": 2, "accuracy": 9, "knowledge": 8},
    "Philosophy for Explorers": {"creativity": 8, "accuracy": 5, "knowledge": 7},
    "Engineering Manual": {"creativity": 3, "accuracy": 10, "knowledge": 9},
    "Storytelling Secrets": {"creativity": 10, "accuracy": 2, "knowledge": 5},
}

In [ ]:
def recommend_book(state: BookState) -> BookState:
    weights = state["weights"]
    scores = {}
    for book, traits in book_database.items():
        score = sum(weights.get(k, 0) * traits.get(k, 0) for k in traits)
        scores[book] = score
    best_book = max(scores, key=scores.get)
    print(f"[Recommender] Scores: {scores}")
    return {**state, "recommended_book": best_book}

def simulate_feedback(state: BookState) -> BookState:
    book = state["recommended_book"]
    prefs = state["preferences"]
    traits = book_database.get(book, {})
    alignment = sum(1 if traits.get(k, 0) >= prefs.get(k, 5) else -1 for k in prefs)
    feedback = "good" if alignment >= 0 else "bad"
    print(f"[Feedback] User rated '{book}' as: {feedback}")
    return {**state, "feedback": feedback}

def update_weights(state: BookState) -> BookState:
    feedback = state["feedback"]
    book = state["recommended_book"]
    traits = book_database[book]
    adjustment = 1 if feedback == "good" else -1
    weights = state["weights"].copy()
    for k in weights:
        delta = 0.1 * adjustment * traits.get(k, 0)
        weights[k] += delta
        weights[k] = max(0.0, min(10.0, weights[k]))
    print(f"[Updater] New weights: {weights}")
    return {**state, "weights": weights}

In [ ]:
# Create the LangGraph
builder = StateGraph(BookState)
builder.add_node("recommend", RunnableLambda(recommend_book))
builder.add_node("feedback", RunnableLambda(simulate_feedback))
builder.add_node("update", RunnableLambda(update_weights))

builder.set_entry_point("recommend")
builder.add_edge("recommend", "feedback")
builder.add_edge("feedback", "update")
builder.add_conditional_edges(
    "update",
    lambda state: "end",  # Stop after one cycle per round
    {"end": END}
)

graph = builder.compile()

# Initial user preference state
initial_state = {
    "preferences": {"creativity": 8, "accuracy": 4, "knowledge": 6},
    "recommended_book": "",
    "feedback": "",
    "weights": {"creativity": 5.0, "accuracy": 5.0, "knowledge": 5.0},
}

# Run 5 rounds
state = initial_state
for i in range(5):
    print(f"\n=== Round {i+1} ===")
    state = graph.invoke(state, config={"recursion_limit": 100})
    recommended_book = state['recommended_book']
    traits = book_database[recommended_book]
    explanation_prompt = (
        f"The book '{recommended_book}' was selected because it aligns well with user preferences: "
        f"{state['preferences']}. The book's traits are: {traits}. "
        f"Explain briefly in one line why this book is a good match."
    )
    explanation = llm.invoke(explanation_prompt).content.strip()
    print(f"[Final Recommendation] {recommended_book}")
    print(f"[Reason] {explanation}")